In [ ]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [ ]:
!wget https://monujohn.com/EEGdata/WLDataAll.mat

--2021-08-04 07:49:42--  https://monujohn.com/EEGdata/WLDataAll.mat
Resolving monujohn.com (monujohn.com)... 198.136.51.114
Connecting to monujohn.com (monujohn.com)|198.136.51.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42375907 (40M)
Saving to: ‘WLDataAll.mat’

WLDataAll.mat       100%[===================>]  40.41M  13.6MB/s    in 3.0s    

2021-08-04 07:49:45 (13.6 MB/s) - ‘WLDataAll.mat’ saved [42375907/42375907]



In [ ]:
annots = loadmat('WLDataAll.mat')

In [ ]:
data = annots['data']

In [ ]:
data_transpose = data[0].T
print('Original shape:', data_transpose.shape)

data_squared = np.square(data_transpose)
data_sum = data_squared.sum(axis=1)


print('New shape:', data_sum.shape)

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


data_normalized = NormalizeData(data_sum)

Original shape: (360, 512)
New shape: (360,)


In [ ]:
datapoint_array = []

for datapoint in data:
  datapoint = datapoint.T
  
  data_sum_squared = np.square(datapoint)
  
  data_sum = data_sum_squared.sum(axis=1)
  
  data_normalized = NormalizeData(data_sum)
  datapoint_array.append(data_normalized)


datapoint_array = np.array(datapoint_array)

print('Latest array shape:', datapoint_array.shape)


Latest array shape: (62, 360)


In [ ]:
X = datapoint_array
Y = annots['label']

Y = np.where(Y == 1, 0, Y)
Y = np.where(Y == 2, 1, Y)

print('X shape:', X.shape)
print('Y shape:', Y.shape)

X shape: (62, 360)
Y shape: (1, 360)


In [ ]:
class ActivationDoesNotExist(Exception):
    """Valid activations are sigmoid, tanh, and relu, provided as a string"""
    pass

class InputDimensionNotCorrect(Exception):
    """Need to specify input dimension, i.e. input shape into the first layer"""
    pass

class LossFunctionNotDefined(Exception):
    """Loss function in cost() method not defined"""
    pass


class DenseLayer:
    def __init__(self, inputDimension, units, activation='', randomMultiplier=0.01):
        self.weights, self.bias = self.initialize(inputDimension, units, randomMultiplier)
        if activation == 'sigmoid':
            self.activation = activation
            self.activationForward = self.sigmoid
            self.activationBackward = self.sigmoidGrad
        elif activation == 'relu':
            self.activation = activation
            self.activationForward = self.relu
            self.activationBackward = self.reluGrad
        elif activation == 'tanh':
            self.activation = activation
            self.activationForward = self.tanh
            self.activationBackward = self.tanhGrad
        elif activation != '':
            raise ActivationDoesNotExist
        else:
            self.activation = 'none'
            self.activationForward = self.linear
            self.activationBackward = self.linear

    def initialize(self, nx, nh, randomMultiplier):
        weights = randomMultiplier * np.random.randn(nh, nx)
        bias = np.zeros([nh, 1])
        return weights, bias

    def sigmoid(self, Z):
        """
        Sigmoid activation function
        """
        A = 1 / (1 + np.exp(-Z))
        return A
        
    def sigmoidGrad(self, dA):
        """
        Differential of sigmoid function with chain rule applied
        """
        s = 1 / (1 + np.exp(-self.prevZ))
        dZ = dA * s * (1 - s)
        return dZ
    
    
    def relu(self, Z):
        """
        Relu activation function
        """
        A = np.maximum(0, Z)
        return A
  
    def reluGrad(self, dA):
        """
        Differential of relu function with chain rule applied
        """
        s = np.maximum(0, self.prevZ)
        dZ = (s>0) * 1 * dA
        return dZ 

        
    def tanh(self, Z):
        """
        Tanh activation function
        """
        A = np.tanh(Z)
        return A

    def tanhGrad(self, dA):
        """
        Differential of tanh function with chain rule applied
        """
        s = np.tanh(self.prevZ)
        dZ = (1 - s**2) * dA
        return dZ


    def linear(self, Z):
        """
        Placeholder when no activation function is used
        """
        return Z
        
    
    def forward(self, A):
        """
        Forward pass through layer
          A: input vector
        """
        Z = np.dot(self.weights, A) + self.bias
        self.prevZ = Z
        self.prevA = A
        A = self.activationForward(Z)
        return A
    
    
    def backward(self, dA):
        """
        Backward pass through layer
          dA: previous gradient
        """
        dZ = self.activationBackward(dA)
        m = self.prevA.shape[1]
        self.dW = 1 / m * np.dot(dZ, self.prevA.T)
        self.db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        prevdA = np.dot(self.weights.T, dZ)
        return prevdA
    
    
    def update(self, learning_rate):
        """
        Update weights using gradients from backward pass
          learning_rate: the learning rate used in the gradient descent
        """
        self.weights = self.weights - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db

        
    def outputDimension(self):
        """
        Returns the output dimension for the next layer
        """
        return len(self.bias)

    
    def __repr__(self):
        """
        Used to print a pretty summary of the layer
        """
        act = 'none' if self.activation == '' else self.activation
        return f'Dense layer (nx={self.weights.shape[1]}, nh={self.weights.shape[0]}, activation={act})'


class NeuralNetwork:
    """
    Neural Network structure that holds our layers
    """
    
    def __init__(self, loss='cross-entropy', randomMultiplier = 0.01):
        """
        Constructor:
          loss: the loss function. Two are defined:
             - 'cross-entropy' and 'mean-square-error'
          randomMultiplier: multiplier applied to the random weights during initialization
        """
        self.layers=[]
        self.randomMultiplier = randomMultiplier
        if loss=='cross-entropy':
            self.lossFunction = self.crossEntropyLoss
            self.lossBackward = self.crossEntropyLossGrad
        elif loss=='mean-square-error':
            self.lossFunction = self.meanSquareError
            self.lossBackward = self.meanSquareErrorGrad
        else:
            raise LossFunctionNotDefined
        self.loss=loss


    def addLayer(self, inputDimension=None, units=1, activation=''):
        """
        Adds a Dense layer to the network:
          inputDimension: required when it is the first layer. otherwise takes dimensions of previous layer.
          units: number of neurons in the layer
          activation: activation function: valid choices are: 'sigmoid', 'tanh', 'relu', ''
        """
        if (inputDimension is None):
            if (len(self.layers)==0):
                raise InputDimensionNotCorrect
            inputDimension=self.layers[-1].outputDimension()
        layer = DenseLayer(inputDimension, units, activation, randomMultiplier= self.randomMultiplier)
        self.layers.append(layer)

    def crossEntropyLoss(self, Y, A, epsilon=1e-15):
        """
        Cross Entropy loss function
          Y: true labels
          A: final activation function (predicted labels)
          epsilon: small value to make minimize chance for log(0) error
        """
        m = Y.shape[1]
        loss = -1 * (Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon))
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
            
    def crossEntropyLossGrad(self, Y, A):
        """
        Cross Entropy loss Gradient
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = -(np.divide(Y, A) - np.divide(1 - Y, 1 - A))
        return dA
    
    
    def meanSquareError(self, Y, A):
        """
        Mean square error loss function
          Y: true labels
          A: final activation function (predicted labels)
        """
        loss = np.square(Y - A)
        m = Y.shape[1]
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
    
    def meanSquareErrorGrad(self, Y, A):
        """
        Mean square error loss gradient
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = -2 * (Y - A)
        return dA

    
    def cost(self, Y, A):
        """
        Cost function wrapper
          Y: true labels
          A: final activation function (predicted labels)
        """
        return self.lossFunction(Y, A)

        
    def forward(self, X):
        """
        Forward pass through the whole model.
          X: input vector
        """
        x = np.copy(X)
        for layer in self.layers:
            x = layer.forward(x)
        return x
            
    
    def backward(self, A, Y):
        """
        backward pass through the whole model
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = self.lossBackward(Y, A)
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
    
    
    def update(self, learning_rate=0.01):
        """
        Update weights and do a step of gradient descent for the whole model.
          learning_rate: learning_rate to use
        """
        for layer in self.layers:
            layer.update(learning_rate)
    
    
    def __repr__(self):
        """
        Pretty print the model
        """
        layrepr = ['  ' + str(ix+1)+' -> ' + str(x) for ix, x in enumerate(self.layers)]
        return '[\n' + '\n'.join(layrepr) + '\n]'
   
    
    def numberOfParameters(self):
        """
        Print number of trainable parameters in the model
        """
        n = 0
        for layer in self.layers:
            n += np.size(layer.weights) + len(layer.bias)
        print(f'There are {n} trainable parameters in the model.')



def roundValue(A):
    return np.uint8( A > 0.5)

def accuracy(yhat, Y):
    return round(np.sum(yhat==Y) / len(yhat.flatten()) * 1000) / 10

In [ ]:
np.random.seed(1)

model = NeuralNetwork()
model.addLayer(inputDimension=62, units=1, activation='sigmoid')
model

[
  1 -> Dense layer (nx=62, nh=1, activation=sigmoid)
]

In [ ]:
num_iterations = 1000000
for ix in range(num_iterations):
    A = model.forward(X)
    model.backward(A, Y)
    model.update()
    if ix % 10000 == 0:
        yhat = roundValue(A)
        print('cost:', model.cost(Y, A), f'\taccuracy: {accuracy(yhat, Y)}%')

cost: 0.6159966195103541 	accuracy: 65.0%
cost: 0.6157774137979796 	accuracy: 65.0%
cost: 0.6155603418650166 	accuracy: 65.0%
cost: 0.6153453636213386 	accuracy: 65.0%
cost: 0.6151324400390877 	accuracy: 64.7%
cost: 0.6149215331162987 	accuracy: 64.7%
cost: 0.614712605842079 	accuracy: 64.7%
cost: 0.614505622163264 	accuracy: 64.7%
cost: 0.6143005469524708 	accuracy: 64.7%
cost: 0.6140973459774802 	accuracy: 64.7%
cost: 0.613895985871878 	accuracy: 64.4%
cost: 0.6136964341068953 	accuracy: 64.2%
cost: 0.6134986589643832 	accuracy: 64.2%
cost: 0.613302629510873 	accuracy: 64.2%
cost: 0.6131083155726627 	accuracy: 63.9%
cost: 0.6129156877118852 	accuracy: 63.9%
cost: 0.6127247172035086 	accuracy: 63.9%
cost: 0.6125353760132252 	accuracy: 63.9%
cost: 0.6123476367761883 	accuracy: 64.2%
cost: 0.6121614727765571 	accuracy: 64.2%
cost: 0.6119768579278119 	accuracy: 64.2%
cost: 0.6117937667538061 	accuracy: 64.2%
cost: 0.6116121743705192 	accuracy: 64.2%
cost: 0.6114320564684832 	accuracy: 64